In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import accuracy
import torch
import torch.nn as nn
from fastai.callbacks.hooks import *
from fastai.callback import *

In [ ]:
def data_learner_init(PATH, sz, tfms, normalize_stats, model_load_name):
    data = ImageDataBunch.from_folder(PATH, ds_tfms=tfms, size=sz)
    data.normalize(normalize_stats)
    print('Data Loaded and Normalized')
    
    learn = cnn_learner(data, models.resnet101, metrics=accuracy)
    print('Model initialized')
    if model_load_name:
        learn.load('unfreeze101-epoch-1-meanstdnorm')
        print('Moel loaded')
    
    return data, learn

In [ ]:
PATH = "/project/DSone/as3ek/data/patches/1000/gannorm_seem_cinn_256/"
sz = 256
tfms = get_transforms(do_flip=True, flip_vert=True, max_zoom=1.1)
normalize_stats = data.batch_stats()
model_load_name = 'unfreeze101-epoch-1-meanstdnorm' # False if none

In [ ]:
data, learn = data_learner_init(PATH, sz, tfms, normalize_stats, model_load_name)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
def hook_output (module:nn.Module, detach:bool=True, grad:bool=False)->Hook:
    "Return a `Hook` that stores activations of `module` in `self.stored`"
    return Hook(module, _hook_inner, detach=detach, is_forward=not grad)

In [ ]:
m = learn.model.eval()
im,cl = learn.data.dl().dataset[1]
cl = int(cl)
xb,_ = data.one_item(im, detach=False, denorm=False, cpu=False) #put into a minibatch of batch size = 1

with hook_output(m[0]) as hook_a: 
    with hook_output(m[0], grad=True) as hook_g:
        preds = m(xb)
        preds[0,int(cl)].backward() 
acts  = hook_a.stored[0].cpu() #activation maps
if (acts.shape[-1]*acts.shape[-2]) >= 16:
    grad = hook_g.stored[0][0].cpu()
    grad_chan = grad.mean(1).mean(1)
    mult = F.relu(((acts*grad_chan[...,None,None])).sum(0))
    if True:
        xb_im = Image(xb[0])
        _,ax = plt.subplots()
        sz = list(xb_im.shape[-2:])
        xb_im.show(ax)
        ax.imshow(mult, alpha=0.4, extent=(0,*sz[::-1],0),
          interpolation='bilinear', cmap='magma')

In [ ]:
torch.mean(xb)

In [ ]:
data.normalize(data.batch_stats())

In [ ]:
imagenet_stats

In [ ]:
data.batch_stats()

In [ ]:
torch.mean((data.one_batch()[0] - torch.Tensor([0.7406, 0.5687, 0.6999])[...,None,None]) / torch.Tensor([0.1947, 0.2706, 0.1987])[...,None,None])

In [ ]:
torch.Tensor([0.485, 0.456, 0.406])

In [ ]:
denormalize(im)